In [1]:
import asyncio
import random
from functools import wraps
from botocore.exceptions import ClientError


In [2]:
def retry_when_error(max_retries=4, base_delay=1, jitter=0.1):
    """
    A decorator to retry an asynchronous function when an exception occurs.

    Parameters:
    max_retries (int): The maximum number of retries before giving up. Default is 4.
    base_delay (int or float): The base delay between retries in seconds. Default is 1 second.
    jitter (float): The maximum random jitter to add to the delay to avoid thundering herd problem. Default is 0.1 seconds.

    Returns:
    function: A wrapped function that will be retried upon failure.

    Example:
    @retry_when_error(max_retries=3, base_delay=2, jitter=0.5)
    async def my_function():
        # function implementation
    """

    def decorator(func):
        @wraps(func)
        async def wrapper(*args, **kwargs):
            retries = 0
            while retries < max_retries:
                try:
                    return await func(*args, **kwargs)
                except ClientError as e:
                    # Retry on ThrottlingException
                    if e.response['Error']['Code'] == 'ThrottlingException':
                        if retries == max_retries - 1:
                            print(f"Failed after {retries + 1} retries")
                            raise
                        retries += 1
                        delay = (base_delay * 2**retries) + (random.random() * jitter)
                        print(f"Retrying {retries}/{max_retries} with delay {delay:.3f} after error: {e}")
                        await asyncio.sleep(delay)
                        
                    # Raise other exceptions
                    else:
                        raise
                        
            return await func(*args, **kwargs)

        return wrapper

    return decorator


In [7]:
# test it out
@retry_when_error()
async def test(arg1, kwarg1=None):
    rv = random.random()
    if rv < 0.4:
        raise ClientError(
            {'Error': {
                'Code': 'ThrottlingException',
                'Message': 'This is a test error',
                }
            }, 
            "InvokeModel"
        )
    elif rv < 0.55:
        raise ClientError(
            {'Error': {
                'Code': 'SomeOtherException',
                'Message': 'This is the other test error',
                }
            }, 
            "OtherService"
        )
    
    return f"with normal return {arg1} and {kwarg1}"

In [8]:
for i in range(10):
    print(f"trial {i} {await test('arg1', kwarg1='kwarg1')}")

trial 0 with normal return arg1 and kwarg1


ClientError: An error occurred (SomeOtherException) when calling the OtherService operation: This is the other test error